In [1]:
import matplotlib.pyplot as plt
import numpy as np

plt.style.use('default')

plt.rcParams['font.family'] = 'DejaVu Serif'
plt.rcParams['font.size'] = 18
plt.rcParams['axes.linewidth'] = 2

Palette = 'magma'
CycPalette = 'Greys'
DivPalette = 'RdYlGn'

FigSize = (20,12)
FigSizeMono = (20,6)
DPI = 72

In [2]:
import ipywidgets as widgets

In [3]:
a = widgets.FloatSlider(
    value=7.5,
    min=0,
    max=10.0,
    step=0.1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

import IPython

In [18]:
out = widgets.Output(layout={'border': '5px solid white','height': '500px','width': '300px'})

out

Output(layout=Layout(border='5px solid white', height='500px', width='300px'))

In [17]:
with out:
    for i in range(10):
        print(i, 'Hello world!')

In [14]:
from IPython.display import YouTubeVideo

out = widgets.Output(layout={'border': '1px solid black'})
out.append_stdout('Output appended with append_stdout')
out

Output(layout=Layout(border='1px solid black'), outputs=({'output_type': 'stream', 'name': 'stdout', 'text': '…

In [15]:

with out:
    display(widgets.IntSlider())